In [1]:
from ultralytics import YOLO

In [6]:
yolo_model_path = '/Users/francesco/repos/computer_vision_project/saved_models/yolo_trained/yolo11n_float32.tflite'
img_path = '/Users/francesco/repos/computer_vision_project/malignant_test_image.jpg'

model = YOLO(yolo_model_path, task='detect')


results = model.predict(
    source=str(img_path),
    imgsz=608,
    conf=0.5,
    save=False,
    device='cpu',
    verbose=False
)
print("Prediction Results:")
for result in results:
    print(f"Bounding Boxes: {result.boxes.xyxy}")  # Bounding box coordinates
    print(f"Confidences: {result.boxes.conf}")   # Confidence scores
    print(f"Classes: {result.boxes.cls}")

Loading /Users/francesco/repos/computer_vision_project/saved_models/yolo_trained/yolo11n_float32.tflite for TensorFlow Lite inference...
Prediction Results:
Bounding Boxes: tensor([[ 627.2402,  447.6378, 2266.2083, 2363.0325]])
Confidences: tensor([0.8356])
Classes: tensor([0.])


In [7]:
import tensorflow as tf

# Load the TFLite model
interpreter = tf.lite.Interpreter(model_path='/Users/francesco/repos/computer_vision_project/saved_models/skc_model.tflite')
interpreter.allocate_tensors()

# Get input details
input_details = interpreter.get_input_details()
print("Input details:", input_details)

# Get output details
output_details = interpreter.get_output_details()
print("Output details:", output_details)


Input details: [{'name': 'x', 'index': 0, 'shape': array([  1, 224, 224,   3], dtype=int32), 'shape_signature': array([ -1, 224, 224,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
Output details: [{'name': 'Identity', 'index': 298, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


In [8]:
import numpy as np
import tensorflow as tf
from PIL import Image

# Load TFLite model
interpreter = tf.lite.Interpreter(model_path='/Users/francesco/repos/computer_vision_project/saved_models/skc_model.tflite')
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Load and preprocess the image
image = Image.open('/Users/francesco/repos/computer_vision_project/malignant_test_image.jpg').resize((224, 224))
input_data = np.asarray(image).astype('float32') / 255.0  # Normalize to [0, 1]
input_data = np.expand_dims(input_data, axis=0)  # Add batch dimension (shape: [1, 224, 224, 3])

# Check input tensor shape
print("Input tensor shape:", input_details[0]['shape'])  # Should print [1, 224, 224, 3]

# Run inference
interpreter.set_tensor(input_details[0]['index'], input_data)  # Pass 4D tensor (not flattened)
interpreter.invoke()

# Get output
output = interpreter.get_tensor(output_details[0]['index'])
print("Model output:", output)  # Prints the risk score or prediction


ValueError: Cannot set tensor: Dimension mismatch. Got 1 but expected 4 for input 0.